In [2]:
import os
import librosa
import numpy as np
import pandas as pd
from tqdm import tqdm
import tsfel 

# --- Import untuk Seleksi Fitur ---
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

# Tentukan path ke DATASET AUGMENTASI Anda
augmented_path = "C:\\Dokumen\\PSD\\dataset\\voice_augmented"
categories = ["buka", "tutup"]
all_data_dfs = [] 

# Siapkan konfigurasi TSFEL
try:
    cfg = tsfel.get_features_by_domain()
except Exception as e:
    print(f"Error saat memuat konfigurasi TSFEL: {e}")
    cfg = None

print(f"Memulai ekstraksi fitur TSFEL dari {augmented_path}...")

def extract_tsfel_features(file_path, cfg):
    """Memuat file audio dan mengekstrak fitur TSFEL."""
    try:
        audio, sr = librosa.load(file_path, sr=None)
        
        # Tambahkan verbose=0 untuk mematikan progress bar internal TSFEL
        features_df = tsfel.time_series_features_extractor(cfg, audio, fs=sr, verbose=0) 
        return features_df
        
    except Exception as e:
        print(f"Error memproses {file_path}: {e}")
        return None

# Pastikan cfg berhasil dibuat sebelum melanjutkan
if cfg is not None:
    # Proses semua file
    for category in categories:
        category_path = os.path.join(augmented_path, category)
        files = [f for f in os.listdir(category_path) if f.endswith('.wav')]
        
        for file_name in tqdm(files, desc=f"Memproses {category}"):
            file_path = os.path.join(category_path, file_name)
            features_df = extract_tsfel_features(file_path, cfg)
            
            if features_df is not None:
                features_df['label'] = category
                all_data_dfs.append(features_df)

    print("\nEkstraksi fitur selesai.")

    # Gabungkan semua DataFrame menjadi satu
    if all_data_dfs:
        data = pd.concat(all_data_dfs, ignore_index=True)
        print(f"Data asli diekstrak: {data.shape[0]} baris, {data.shape[1]} kolom (termasuk label)")

        # --- 1. Pembersihan Data (PENTING untuk TSFEL) ---
        # TSFEL dapat menghasilkan NaN jika fitur tidak dapat dihitung
        # Opsi 1: Hapus kolom yang mengandung NaN
        # data.dropna(axis=1, inplace=True)
        
        # Opsi 2: Isi NaN dengan 0 (lebih umum)
        data.fillna(0, inplace=True)
        
        # Hapus kolom yang nilainya konstan (tidak ada varians)
        # karena tidak berguna untuk model
        non_variant_cols = [col for col in data.columns if col != 'label' and data[col].nunique() == 1]
        if non_variant_cols:
            print(f"Menghapus {len(non_variant_cols)} kolom konstan...")
            data.drop(columns=non_variant_cols, inplace=True)
            
        print(f"Data setelah pembersihan: {data.shape[0]} baris, {data.shape[1]} kolom")

        # --- 2. Pisahkan Fitur (X) dan Label (y) ---
        X = data.drop('label', axis=1)
        y = data['label']

        # --- 3. Encode Label (Ubah "buka", "tutup" menjadi 0, 1) ---
        le = LabelEncoder()
        y_encoded = le.fit_transform(y)

        # --- 4. Inisialisasi Model untuk Seleksi Fitur ---
        # Kita gunakan Random Forest untuk mengukur "importance" setiap fitur
        rf_clf = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)

        # Inisialisasi pemilih fitur (SelectFromModel)
        # threshold="median" akan memilih fitur yang pentingnya > median
        # Anda juga bisa set max_features=50 untuk memilih 50 fitur teratas
        # selector = SelectFromModel(rf_clf, threshold="median") 
        selector = SelectFromModel(rf_clf, max_features=50, threshold=-np.inf) # Ambil 50 fitur terbaik

        print("\nMemulai seleksi fitur (training Random Forest)...")
        
        # Latih selector
        selector.fit(X, y_encoded)
        
        print("Seleksi fitur selesai.")

        # --- 5. Dapatkan Fitur yang Terpilih ---
        selected_mask = selector.get_support()
        selected_features = X.columns[selected_mask]
        
        print(f"Fitur terpilih: {len(selected_features)} dari {len(X.columns)} fitur asli.")
        # print("Daftar fitur terpilih:", list(selected_features)) # Uncomment jika ingin lihat

        # --- 6. Buat DataFrame Baru Hanya dengan Fitur Terpilih ---
        X_selected = X[selected_features]
        
        # Buat DataFrame akhir
        data_final = X_selected.copy()
        data_final['label'] = y # Tambahkan kembali label asli ("buka", "tutup")
        
        print(data_final.head())

        # --- 7. Simpan ke CSV Baru ---
        output_filename = "voice_features_selected.csv"
        data_final.to_csv(output_filename, index=False)
        
        print(f"\nBerhasil! Data fitur terpilih ({len(data_final)} baris, {data_final.shape[1]} kolom) telah disimpan ke {output_filename}")

    else:
        print("Tidak ada data yang berhasil diekstrak.")
else:
    print("Ekstraksi fitur dibatalkan karena konfigurasi TSFEL gagal dimuat.")

Memulai ekstraksi fitur TSFEL dari C:\Dokumen\PSD\dataset\voice_augmented...


Memproses tutup: 100%|██████████| 100/100 [01:42<00:00,  1.03s/it]



Ekstraksi fitur selesai.
Data asli diekstrak: 200 baris, 157 kolom (termasuk label)
Data setelah pembersihan: 200 baris, 157 kolom

Memulai seleksi fitur (training Random Forest)...
Seleksi fitur selesai.
Fitur terpilih: 50 dari 156 fitur asli.
   0_Autocorrelation  0_Average power  0_Centroid  0_ECDF Percentile Count_0  \
0               19.0      1142.246928    1.552798                    21876.0   
1               24.0       876.867916    0.831794                    16971.0   
2               18.0      1223.100250    1.026031                    18432.0   
3               19.0      1082.023471    1.371756                    19251.0   
4               22.0      1242.740390    1.025401                    27648.0   

   0_ECDF Percentile Count_1  0_ECDF Percentile_0  0_ECDF Percentile_1  \
0                    87505.0            -0.019836             0.020325   
1                    67885.0            -0.034943             0.025726   
2                    73728.0            -0.020905  